 # Homework 2

## 2.1

In [99]:
from collections import Counter
import numpy

# Define the corpuses
spam_corpus = [["I", "am", "spam", "spam", "I", "am"], ["I", "do", "not", "like", "that", "spamiam"]]
ham_corpus = [["do", "i", "like", "green", "eggs", "and", "ham"], ["i", "do"]]

def make_spam_filter(good_corpus, bad_corpus):
    # Flatten the lists and lowercase the tokens in both corpuses
    good_corpus_flat = list(map(lambda x: x.lower(), numpy.concatenate(good_corpus)))
    bad_corpus_flat = list(map(lambda x: x.lower(), numpy.concatenate(bad_corpus)))
    # Create the occurence hashes for each corpus
    good_map = Counter(good_corpus_flat)
    bad_map = Counter(bad_corpus_flat)

    # Token probability function taken from reading
    def calc_token_probability(token):
        g = 2 * (good_map.get(token) or 0)
        b = bad_map.get(token) or 0
        ngood = len(good_map)
        nbad = len(bad_map)
        if g + b > 1:
            return max(0.01, min(0.99, min(1.0, b / nbad) / 
                                 (min(1.0, g / ngood) + min(1.0, b/ nbad))))
        else:
            return 0

    # Table of all the token probabilities
    token_probability = dict(map(
        lambda x: (x, calc_token_probability(x)), good_corpus_flat + bad_corpus_flat
    ))
    # Show the probability table
    print(token_probability)
    
    # Return the token probability or neutral
    def get_token_probability(token):
        return token_probability.get(token) or 0.5
    
    # Probability funtion from the reading
    def spam_probability(message):
        probs = list(map(get_token_probability, message))
        prod = numpy.prod(probs)
        return prod / (prod + numpy.prod(list(map(lambda x: 1 - x, probs))))
        
    
    return spam_probability

# Create the spam function
spam_prob = make_spam_filter(ham_corpus, spam_corpus)

print("Spam probability:", spam_prob("i like ham".split()))
print("Spam probability:", spam_prob("i am ham i am eggs".split()))
print("Spam probability:", spam_prob("I am spam spamiam".split()))

{'do': 0.1794871794871795, 'i': 0.39622641509433965, 'like': 0.30434782608695654, 'green': 0.01, 'eggs': 0.01, 'and': 0.01, 'ham': 0.01, 'am': 0.99, 'spam': 0.99, 'not': 0, 'that': 0, 'spamiam': 0}
Spam probability: 0.0028917084685748016
Spam probability: 0.30102389078498265
Spam probability: 0.9998979800040808


The problem makes use of the Bayes rule where we calculate the probability of the cause given the effect. The email is the effect, and it is used to measure the probability of the cause which could be spam or not spam.


## 2.2

In [93]:
# a.
from probability import BayesNet, enumeration_ask, elimination_ask, gibbs_ask

T, F = True, False

weather = BayesNet([
    ('Cloudy', '', 0.5),
    ('Sprinkler', 'Cloudy', {T: 0.1, F: 0.5}),
    ('Rain', 'Cloudy', {T: 0.8, F: 0.2}),
    ('WetGrass', 'Sprinkler Rain', {(T, T): 0.99, (T, F): 0.9, (F, T): 0.9, (F, F): 0})
])


False: 0.517, True: 0.483
False: 0.912, True: 0.088
False: 0.957, True: 0.043
False: 0.014, True: 0.986
False: 0.654, True: 0.346


b. The full joint probability includes 4 random variables, so the joint distribution would contain 2x2x2x2 or 16 entries.

c. There are only two independent value in the Bayesian network

In [100]:
# d.

print(gibbs_ask('Cloudy', dict(), weather).show_approx())
print(gibbs_ask('Sprinkler', dict(Cloudy=T), weather).show_approx())
print(gibbs_ask('Cloudy', dict(Sprinkler=T, Rain=F), weather).show_approx())
print(gibbs_ask('WetGrass', dict(Cloudy=T, Sprinkler=T, Rain=T), weather).show_approx())
print(gibbs_ask('Cloudy', dict(WetGrass=F), weather).show_approx())

False: 0.52, True: 0.48
False: 0.879, True: 0.121
False: 0.951, True: 0.049
False: 0.013, True: 0.987
False: 0.641, True: 0.359
